In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
import requests
import xml.etree.ElementTree as ET
from typing import *
import time
from utils import *
import threading
import queue
import pandas as pd

In [3]:
def make_esearch_call(query: str, api_key: None | str, email: str = "your_email@example.com") -> Tuple[int, str, str]:
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"

    params = {
        "db": "pubmed",
        "term": query,
        "usehistory": "y",
        "retmode": "json",
        "retmax": 0,
        "email": email
    }

    if api_key:
        params["api_key"] = api_key

    res = requests.get(url, params=params)
    res.raise_for_status()

    data = res.json()["esearchresult"]

    count = int(data["count"])
    webenv = data["webenv"]
    query_key = data["querykey"]

    print(f"Total results: {count}")
    
    return count, webenv, query_key

In [4]:
NCBI_EMAIL = os.getenv("NCBI_API_KEY")
NCBI_API_KEY = os.getenv("email")
query = "covid-19 AND 2024[dp] AND humans[MeSH Terms]"

count, webenv, query_key = make_esearch_call(query, NCBI_EMAIL,NCBI_API_KEY)

Total results: 40507


In [ ]:
RETMAX=1000

params = {
            "db": "pubmed",
            "query_key": query_key,
            "WebEnv": webenv,
            "retstart": 0,
            "retmax": RETMAX,
            "retmode": "xml",
            "email": NCBI_EMAIL,
            "api_key": None
        }
res=None
try:
    res = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi", params=params)
    res.raise_for_status()

    filename = f"pubmed_batch_{0}_{0 + RETMAX - 1}.xml"

    with open(filename, "w", encoding="utf-8") as f:
        f.write(res.text)

    print(f"[EF-{1}] Fetched records {0}-{0 + RETMAX - 1}")

except Exception as e:
    print(f"[EF-{1}] Error at start={0}: {e}")
finally:
    #q.task_done()
    #time.sleep(1.0 / RATE_LIMIT)  # To respect NCBI limit

[EF-1] Fetched records 0-999


NameError: name 'RATE_LIMIT' is not defined

In [6]:
from utils import *
root = ET.fromstring(res.text)
articles = root.findall(".//PubmedArticle")

In [7]:
out=[]
for article in articles:
        parsed = parsing_an_article(article)
        out.append(parsed)

In [8]:
import pandas as pd

def save_to_csv(data_list, filename):
    df = pd.DataFrame(data_list)

    # Convert sets to semicolon-separated strings
    df["NonAcademicAuthors"] = df["NonAcademicAuthors"].apply(lambda x: ", ".join(x))
    df["CompanyAffiliations"] = df["CompanyAffiliations"].apply(lambda x: ", ".join(x))

    # Save to CSV
    df.to_csv(filename, index=False)

In [ ]:
save_to_csv(out,"default.csv")

In [13]:
df=pd.read_csv('default.csv')
df.head(10)

,PMID,Title,PublicationDate,NonAcademicAuthors,CompanyAffiliations,CorrespondingAuthorEmail
0,40599452,Safflower-Derived Cationic Lipid Nanoparticles...,29-6-2024,"H Nasr Isfahani, Z Sharifnia, A Moradi, A A Ha...","Biotechnology Department, Behyaar Sanaat Sepah...",NaN
1,40599443,"Assessment of COVID-19 Vaccine Knowledge, Atti...",26-12-2023,NaN,NaN,NaN
2,40599434,A Concise Review of Major Challenges in the Va...,2-12-2024,NaN,"Department of Medical Biotechnology, School of...",NaN
3,40587612,Stellate Ganglion Block for Treating Post-COVI...,26-9-2024,NaN,NaN,NaN
4,40587610,Use of Neuromodulation to Treat Post-COVID Neu...,10-7-2024,Casey Scott Butler,NaN,NaN
5,40587609,Tibial Nerve Mononeuropathy Post-COVID-19 and ...,10-7-2025,NaN,NaN,NaN
6,40581392,Bridging the gap across research and practice:...,1-7-2025,NaN,NaN,mcbridca@purdue.edu.
7,40561331,"[Fiscal austerity, COVID-19 and metropolises: ...",5-4-2024,"Alessandro Jatobá, Nilson do Rosário Costa, Ra...",NaN,nilsondorosario@gmail.com.
8,40561326,[Prevalence of influenza vaccine before and du...,29-11-2023,NaN,NaN,renatapfontanella@hotmail.com.
9,40561323,[The COVID-19 pandemic and food and nutritiona...,21-9-2023,Juliana Márcia Macedo Lopes,NaN,mariaclara.rabello@estudante.ufjf.br.
